In [1]:
import os
import math

In [36]:
filepath = "./blaze/src/b.txt"

In [3]:
x_o, y_o, z_o = -305.923, -2.358, -798.047
x_x, y_x, z_x = -316.039, -178.714, -530.861
x_y, y_y, z_y = 295.871, 0.771, -794.537

In [4]:
with open(filepath, 'r') as f:
    lines = f.readlines()

In [5]:
points = [line.split()[:3] for line in lines]
for point in points:
    point[0] = float(point[0]) - x_o
    point[1] = float(point[1]) - y_o
    point[2] = float(point[2]) - z_o

In [6]:
with open("./blaze/dst/a_differential.txt", 'w') as f:
    for i in range(len(points)):
        f.write(str(points[i][0]) + " " 
                + str(points[i][1]) + " "
                + str(points[i][2]) + "\n"  )

In [7]:
y_r = y_o - y_x
z_r = z_o - z_x
r = abs(math.sqrt((y_r)**2 + (z_r)**2))
cos = y_r / r
sin = z_r / r

In [8]:
#x軸に対して負の回転
res = []
for point in points:
    x, y, z = point
    X = x
    Y = (y*cos) + (z*sin)
    Z = (-y*sin) + (z*cos)
    res.append([X, Y, Z])

In [9]:
x_r = x_o - x_x
y_r = y_o - y_x
r = abs(math.sqrt((x_r)**2 + (y_r)**2))
cos = x_r / r
sin = y_r / r

In [31]:
#z軸に対して正の回転
res_z = []
deg = 90
cos = math.cos(math.radians(deg))
sin = math.sin(math.radians(deg))
for point in res:
    x, y, z = point
    Z = z
    X = (x*cos) + (-y*sin)
    Y = (x*sin) + (y*cos)
    res_z.append([X, Y, Z])

In [32]:
with open("./blaze/dst/a_rotation_x_z.txt", 'w') as f:
    for i in range(len(res)):
        f.write(str(res_z[i][0]) + " "
                + str(res_z[i][1]) + " "
                + str(res_z[i][2]) + "\n"  )

In [117]:
def read_blazedata(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()
    points = [line.split()[:3] for line in lines]
    
    return points

def write_data(output_path, _res):
    with open(output_path, 'w') as f:
        for i in range(len(_res)):
            f.write(str(_res[i][0]) + " "
                    + str(_res[i][1]) + " "
                    + str(_res[i][2]) + "\n"  )            

def set_origin(points, origin):
    x, y, z = origin
    for point in points:
        point[0] = float(point[0]) - x
        point[1] = float(point[1]) - y
        point[2] = float(point[2]) - z
        
def calculate_degree(origin, point, axis):
    x_o, y_o, z_o = origin
    x_p, y_p, z_p = point
    if axis == 'x':
        r1 = y_o - y_p
        r2 = z_o - z_p
    elif axis == 'y':
        r1 = z_o - z_p
        r2 = x_o - x_p
    elif axis == 'z':
        r1 = x_o - x_p
        r2 = y_o - y_p
    r = abs(math.sqrt((r1)**2 + (r2)**2))
    cos = r1 / r
    deg = math.degrees(math.acos(cos))
    
    return deg

def rotate_with_x(points, deg, pn):
    '''
    Parameters
    ----------
    pn : 1 or -1
    '''
    _res = []
    cos = math.cos(math.radians(deg))
    sin = math.sin(math.radians(deg))
    for point in points:
        x, y, z = point
        X = x
        Y = (y*cos) + (-z*sin*pn)
        Z = (y*sin*pn) + (z*cos)
        _res.append([X, Y, Z])
        
    return _res      

def rotate_with_z(points, deg, pn):
    ''' 
    Parameters
    ----------
    pn : 1 or -1
    '''
    _res = []
    cos = math.cos(math.radians(deg))
    sin = math.sin(math.radians(deg))
    for point in points:
        x, y, z = point
        Z = z
        X = (x*cos) + (-y*sin*pn)
        Y = (x*sin*pn) + (y*cos)
        _res.append([X, Y, Z])
        
    return _res

def scaling(points, ratio):
    for point in points:
        point[0] = point[0] * ratio
        point[1] = point[1] * ratio
        point[2] = point[2] * ratio
        
def thinning(points, _range, axis):
    daxis = {"x": 0, "y":1, "z":2}
    _res = [point for point in points 
            if _range[0]<=point[daxis[axis]]<=_range[1]]
    
    return _res

def grid(points):
    for point in points:
        point[0] = round(float(point[0]))
        point[1] = round(float(point[1]))
        point[2] = float(point[2])
        
def averaging(points):
    unique_x = set([point[0] for point in points])
    sort_xs = [[point for point in points if point[0]==x] for x in unique_x]
    unique_y = set([point[1] for point in sort_xs[0]])
    sort_ys = [[[point for point in sort_x if point[1]==y] for y in unique_y] for sort_x in sort_xs]

    _average = []
    for x_len in range(len(unique_x)):
        for y_len in range(len(unique_y)):
            s = 0
            for i in range(len(sort_ys[x_len][y_len])):
                s += sort_ys[x_len][y_len][i][2]
               
            if s!=0:
                z_mean = s / len(sort_ys[x_len][y_len])
                _average.append([sort_ys[x_len][y_len][0][0],
                            sort_ys[x_len][y_len][0][1],
                            z_mean])
    
    return _average

In [50]:
points = read_blazedata(filepath)

In [52]:
origin = [-432.486, -76.069, -956.741]
set_origin(points, origin) 

In [81]:
point = [-436.692, -129.157, -871.443]
deg = calculate_degree(origin, point, 'x')
res = rotate_with_x(points, deg, 1)
res = rotate_with_z(res, 175.5, -1)

In [87]:
ratio = 34 / 440.295
scaling(res, ratio)

In [89]:
write_data("./blaze/dst/b_rotate_x.txt", res)

In [92]:
#データの間引き
grid(res)

In [110]:
thz_res = thinning(res, [-4, 10], "z")
thy_res = thinning(thz_res, [-17, 17], "y")
thx_res = thinning(thy_res, [-70, 1], "x")

In [111]:
write_data("./blaze/dst/b_thx.txt", thx_res)

In [118]:
#同じ座標の点を一つにする
ave_res = averaging(thx_res)

In [119]:
write_data("./blaze/dst/b_ave.txt", ave_res)